[View in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/index.ipynb)

# Combinación de bases de datos de artículos científicos
```
URL_base=https://colab.research.google.com/github/blob_file_in_repo
blob_file_in_repo → restrepo/medicion/blob/master/cienciometria/merge.ipynb

```
## UdeA
### Combinar bases de datos bibliográficas UDEA
* [Combibnar W0S+SCP+SCI](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/merge.ipynb) versión detallada
* [Combibnar W0S+SCP+SCI](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/WOS_SCI_SCP.ipynb) versión corta
* [Open Access](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/OpenAccess.ipynb) DOAJ API 
* [Open Access](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/XXX.ipynb)   https://unpaywall.org/ dataset. See https://towardsdatascience.com/interactively-analyse-100gb-of-json-data-with-spark-e018f9436e76
```bash
# Select True Open Access articles
grep -E '"is_oa":\s*true' unpaywall_snapshot_2018-06-21T164548_with_versions.jsonl  >  oa.json
# Select only DOIS values: for True Opena Access articles
echo DOI > oadoi.csv
awk -F","  '{print $1}' oa.json | grep '"doi":' | awk -F '"' '{print $(NF-1)}'   >> oadoi.csv
# Full DOIs
echo DOI > dois.csv
awk -F","  '{print $1}' unpaywall_snapshot_2018-06-21T164548_with_versions.jsonl | grep '"doi":' | awk -F '"' '{print $(NF-1)}'   >> dois.csv
```
Results here:
1. [oadois.csv](https://drive.google.com/file/d/1uipr7JlNhLzZK5W8VymUNwVGGim-AqhC/view?usp=sharing)
1. [dois.csv]()
* TODO: comprobar los APIS de http://blog.impactstory.org/introducting-oadoi/. Por ejemplo
Usar
* http://oadoi.org/10.1103/PhysRevD.98.055016
En lugar de   http:://doi.org/10.1103/PhysRevD.98.055016